In [1]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog as fd

Read in the data as a multi-hierarchical dataframe

In [2]:
root = tk.Tk()
file = fd.askopenfile()
root.destroy()

df = pd.read_excel(file.name, header=[0,1])
df.head()

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 February 1, 2020  \
                  ID              World               Area          Hobbits   
0              C0132       Middle Earth               1523              423   
1              C4974           Tatooine               1091              258   
2              C2967          Gallifrey               1397              210   
3              C9014          Magrathea               1644              274   
4              C7046         Themyscira               1939              413   

                                   April 1, 2020       ... October 1, 2020  \
  Jedi Time Lords Dolphins Amazons       Hobbits Jedi  ...         Hobbits   
0  411        186      439     156           334  372  ...            1254   
1  183        199      154     287           427  675  ...             985   
2  494        338      290     384           313  629  ...            1152   
3  409        458      230     287           637  472  ...            1177   
4  205        352      188     142           643  369  ...            1046   

                                    December 1, 2020                   \
   Jedi Time Lords Dolphins Amazons          Hobbits  Jedi Time Lords   
0  1184       1005     1216     922             1279  1297       1145   
1  1260       1229     1123    1260             1225  1153       1385   
2  1121       1077     1040    1034             1176  1101       1372   
3   926       1012     1195    1130             1195  1135       1307   
4  1251       1017     1186    1158             1101  1185       1251   

                    
  Dolphins Amazons  
0     1196    1342  
1     1367    1152  
2     1394    1234  
3     1369    1415  
4     1320    1379  

[5 rows x 33 columns]

If you want to get a list of columns, you will get a list of tuples (main name, sub-name).  
We want a list of just main name, so iterate over the column names and form a new list of unique column names.  
We want all but the first group of columns...

In [3]:
dates = []
for i in list(df.columns):
    if i[0] not in dates and not i[0].startswith("Unnamed"):
        dates.append(i[0])

We now extract the columns under each main column name and set that as its own dataframe... We use the `globals()` function to create sequential variable names for the different dataframes.

In [4]:
extracted_dfs = []
for i, val in enumerate(dates):
    globals()["df%s" %i] = df[[val]]
    extracted_dfs.append(globals()["df%s" %i])

We need to define two functions...
1. df_flatten will help flatten the multi-hierarchical dataframe into a flat dataframe. We use the `to_records()` method for this.
2. df_organize will move the date column to the start of the dataframe.

In [5]:
def df_flatten(df):
    df = pd.DataFrame(df.to_records())
    df.drop("index",axis="columns",inplace=True)
    return df

def df_organize(df,index):
    date = df[list(df.columns)[index]]
    df = df.drop(['date'],axis=1)
    df.insert(0,'date',date)    
    return df

We shall create a new list of the dataframes. We will process them so that for each dataframe we got:
1. Flatten it
2. Rename the columns
3. Add a date column
4. Move the date column to the beginning of the dataframe

In [6]:
column_names = []
for i in list(df.columns):
    if i[1] not in column_names and not i[0].startswith("Unnamed"):
        column_names.append(i[1])

reorganized_dfs = []
for i, value in enumerate(extracted_dfs):
    value = df_flatten(value)
    value.columns = column_names
    value["date"] = dates[i]
    value = df_organize(value,-1)
    reorganized_dfs.append(value)

We also want to create a dataframe of the first 3 columns of the spreadsheet. So subset it from the main dataframe, then flatten it and rename the columns.

In [7]:
column_names = []
for i in list(df.columns):
    if i[1] not in column_names and i[0].startswith("Unnamed"):
        column_names.append(i[1])

df_first_columns = df.iloc[:,:3]
df_flatten(df_first_columns)
df_first_columns.columns = column_names

Now we want to create a new list of dataframes. These dataframes are concatenations of the first 3 columns and the 4 columns of each of the period dataframes.

In [8]:
dfs_to_concatenate = []
for i, value in enumerate(reorganized_dfs):
    globals()["d%s" %i] = pd.concat([df_first_columns,reorganized_dfs[i]],1)
    dfs_to_concatenate.append(globals()["d%s" %i])
df_final = pd.concat(dfs_to_concatenate,ignore_index=True)

Now we move the date column to the beginning.  
Then we convert it from the given format to the MMM-DD format.

In [9]:
df_final = df_organize(df_final,3)
df_final["date"] = pd.to_datetime(df_final["date"])
df_final["date"] = df_final["date"].apply(lambda x: x.strftime("%b-%d"))

Finally we write the dataframe to file.

In [10]:
df_final.to_excel("../data/worlds census.xlsx",index=False)